In [1]:
from numpy import load
import numpy as np
import pandas as pd
import itertools
import requests
from io import StringIO
import pyarrow.parquet as pq


ModuleNotFoundError: No module named 'pyarrow'

In [ ]:
#data = load('classes_sherlock.npy', allow_pickle=True)
#print(data)

In [ ]:
acceptable_columns = ['address', 'age', 'area', 'birth Date', 'birth Place', 'brand', 'city', 'continent', 'country', 'county', 'currency', 'day', 'duration', 'industry', 'language', 'location', 'manufacturer', 'name', 'nationality', 'order', 'person', 'product', 'range', 'rank', 'region','sales', 'sex', 'state', 'status', 'symbol', 'type', 'year']
acceptable_columns = np.array(acceptable_columns)
# np.save("classes_sherlock.npy", acceptable_columns, allow_pickle=False)


In [6]:
# main dataframes to add onto existing dataframe
new_train = pd.DataFrame(columns = ['type', 'values'])
new_test = pd.DataFrame(columns = ['type', 'values'])
new_val = pd.DataFrame(columns = ['type', 'values'])

# list of urls of csvs
urls = ['https://drive.google.com/file/d/1VBEYn2bV9hUg6Vs97StMMR8NudP--v-D/view?usp=sharing', 'https://drive.google.com/file/d/1XoHPXvb_eAH-90jTSj-1sePVzQtIlw-n/view?usp=sharing',
       'https://drive.google.com/file/d/1QwjBOKHSYbEqZxDeUfVZVOjkgb_xuaUt/view?usp=sharing', 'https://drive.google.com/file/d/1I4btyUubuYR6sgz2YhfEftNXdJuKaPaN/view?usp=sharing',
       'https://drive.google.com/file/d/1X6B-9hRgENR4xAjL1d0m6BmxGZUs6eGF/view?usp=sharing', 'https://drive.google.com/file/d/1DiHNQZUb9icUhkeBvWYyARKYscDPUAyx/view?usp=sharing',
       'https://drive.google.com/file/d/1GhY41caqsjYhTVdUaHqyzp6b-bUJSiX9/view?usp=sharing', 'https://drive.google.com/file/d/1ISLN-fra-7rzfar4b8LvM20YF_UNWC52/view?usp=sharing',
       'https://drive.google.com/file/d/140MKfvrMnIy-sfNlovGmHShEETBagMBw/view?usp=sharing']
# list of types
types = ['naic', 'credit card account numbers', 'credit card account numbers', 'credit card account numbers', 'fips code', 'lei',
        'mcc code', 'phone number', 'zip code']

In [7]:
for i in range(len(urls)):
    url = urls[i]
    file_id = url.split('/')[-2]
    dwn_url='https://drive.google.com/uc?id=' + file_id
    data = pd.read_csv(dwn_url, sep=";", header=None)
    t = types[i]

    # split 0.8 to training, 0.1 to test, and 0.1 to validation
    train_idx = round(len(data)*0.8)
    test_idx = round(len(data)*0.1)
    train_data = data.iloc[:train_idx]
    test_data = data.iloc[train_idx: (train_idx+test_idx)]
    val_data = data.iloc[(train_idx+test_idx):]

    # splitting into chunks of ~five data points
    train_data = train_data.values.tolist()
    train_data = list(itertools.chain(*train_data))
    splitval = train_idx/5
    train_data_split = np.array_split(train_data, splitval)
    train_df = pd.Series(train_data_split, name='values').to_frame()
    train_df['type'] = t
    train_df = train_df.iloc[:,[1,0]]

    splitval = round(test_idx/5)
    test_data = test_data.values.tolist()
    test_data = list(itertools.chain(*test_data))
    test_data_split = np.array_split(test_data, splitval)
    test_df = pd.Series(test_data_split, name='values').to_frame()
    test_df['type'] = t
    test_df = test_df.iloc[:,[1,0]]

    splitval = round(len(val_data)/5)
    val_data = val_data.values.tolist()
    val_data = list(itertools.chain(*val_data))
    val_data_split = np.array_split(val_data, splitval)
    val_df = pd.Series(val_data_split, name='values').to_frame()
    val_df['type'] = t
    val_df = val_df.iloc[:,[1,0]] 
    
    # add to main dataframes
    new_train = new_train.append(train_df, ignore_index=True)
    new_test = new_test.append(test_df, ignore_index=True)
    new_val = new_val.append(val_df, ignore_index=True)

In [10]:
#new_train

,type,values
0,naic,"[713910, 71392, 713920, 71393, 713930]"
1,naic,"[71394, 713940, 71395, 713950, 71399]"
2,naic,"[713990, 72, 721, 7211, 72111]"
3,naic,"[721110, 72112, 721120, 72119, 721191]"
4,naic,"[721199, 7212, 72121, 721211, 721214]"
...,...,...
1844,zip code,"[917, 641, 612, 611, 687]"
1845,zip code,"[646, 692, 692, 674, 687]"
1846,zip code,"[693, 694, 765, 766, 720]"
1847,zip code,"[767, 653, 606, 698, 850]"


In [ ]:
test_df_values = pq.read_table(source="./data/raw/test_values.parquet").to_pandas()
test_df_labels = pq.read_table(source="./data/raw/test_labels.parquet").to_pandas()
test_df_full = test_df_labels.merge(test_df_values, left_index=True, right_index=True)
test_df_full = test_df_full[test_df_full['type'].isin(acceptable_columns)]
# add new testing data
test_df_full = test_df_full.append(new_test, ignore_index=True)
test_df_values = pd.DataFrame(test_df_full, columns=['values'])
test_df_labels = pd.DataFrame(test_df_full, columns=['type'])
len(test_df_full.iloc[5])

In [ ]:
train_df_values = pq.read_table(source="./data/raw/train_values.parquet").to_pandas()
train_df_labels = pq.read_table(source="./data/raw/train_labels.parquet").to_pandas()
train_df_full = train_df_labels.merge(train_df_values, left_index=True, right_index=True)
train_df_full = train_df_full[train_df_full['type'].isin(acceptable_columns)]
# add new training data
train_df_full = train_df_full.append(new_train, ignore_index=True)
train_df_values = pd.DataFrame(train_df_full, columns=['values'])
train_df_labels = pd.DataFrame(train_df_full, columns=['type'])
train_df_full.loc[train_df_full["type"] == "symbol"]

In [ ]:
val_df_values = pq.read_table(source="./data/raw/val_values.parquet").to_pandas()
val_df_labels = pq.read_table(source="./data/raw/val_labels.parquet").to_pandas()
val_df_full = val_df_labels.merge(val_df_values, left_index=True, right_index=True)
val_df_full = val_df_full[val_df_full['type'].isin(acceptable_columns)]
# add new validation data
val_df_full = val_df_full.append(new_val, ignore_index=True)
val_df_values = pd.DataFrame(val_df_full, columns=['values'])
val_df_labels = pd.DataFrame(val_df_full, columns=['type'])
val_df_full

In [ ]:
test_df_labels.to_parquet("./data/raw/test_labels.parquet", engine='fastparquet')
test_df_values.to_parquet("./data/raw/test_values.parquet", engine='fastparquet')

train_df_labels.to_parquet("./data/raw/train_labels.parquet", engine='fastparquet')
train_df_values.to_parquet("./data/raw/train_values.parquet", engine='fastparquet')

val_df_labels.to_parquet("./data/raw/val_labels.parquet", engine='fastparquet')
val_df_values.to_parquet("./data/raw/val_values.parquet", engine='fastparquet')

